In [ ]:
import warnings
warnings.filterwarnings('ignore') # uncomment this line to suppress warnings

import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix, f1_score, cohen_kappa_score

random_state = 42
     

# 1

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Tracce altro drive/30-06-2022/horse-colic.csv',names=[str(i) for i in range(1,29)])
df.shape

(300, 28)

In [ ]:
df = df.drop(['3','25','26','27','28'],axis=1)

In [ ]:
df = df.replace(['?'],np.nan)

In [ ]:
df.isna().sum()

1       1
2       0
4      60
5      24
6      58
7      56
8      69
9      47
10     32
11     55
12     44
13     56
14    104
15    106
16    247
17    102
18    118
19     29
20     33
21    165
22    198
23      1
24      0
dtype: int64

# 2

In [ ]:
target = '23'
df = df.dropna(axis=0, subset=[target])
df.shape

(299, 23)

In [ ]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
col = df.loc[:, df.columns != target].columns
df_predicting = pd.DataFrame(imp_mean.fit_transform(df.loc[:, df.columns != target]), columns =col)

In [ ]:
X = df_predicting
y = df[target]
print(X.shape,y.shape)

(299, 22) (299,)


# 3

In [ ]:
from sklearn.model_selection import train_test_split
ts = 0.3
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size = ts, random_state = random_state)
print(Xtrain.shape,Xtest.shape,)

(209, 22) (90, 22)


In [ ]:
tuned_param_adb = [{'n_estimators':[20,30,40,50],
                   'learning_rate':[0.5,0.75,1,1.25,1.5]}]
clf = GridSearchCV(AdaBoostClassifier(),tuned_param_adb,cv=5,
                           scoring='accuracy', 
                           return_train_score = False,
                           n_jobs = 2, # this allows using multi-cores
                           )

clf.fit(Xtrain, ytrain)
print("Best parameters set found on train set:")
print()
print(clf.best_params_)
y_pred = clf.predict(Xtest)

Best parameters set found on train set:

{'learning_rate': 1, 'n_estimators': 40}


In [ ]:
print(confusion_matrix(ytest, y_pred,normalize='true'))

[[0.78       0.08       0.14      ]
 [0.35714286 0.5        0.14285714]
 [0.58333333 0.08333333 0.33333333]]


In [ ]:
f1_model1 = f1_score(ytest, y_pred, average='macro')
print(f1_model1)

0.5426300112503779


In [ ]:
cohen_kappa_model1 = cohen_kappa_score(ytest, y_pred)
print(cohen_kappa_model1)

0.35265911072362677


# 4

In [ ]:
tuned_param_nb = [{'var_smoothing': [10, 1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-07, 1e-8, 1e-9, 1e-10]}]

clf = GridSearchCV(GaussianNB(),tuned_param_nb,cv=5,
                           scoring='accuracy', 
                           return_train_score = False,
                           n_jobs = 2, # this allows using multi-cores
                           )

clf.fit(Xtrain, ytrain)
print("Best parameters set found on train set:")
print()
print(clf.best_params_)
y_pred_nb = clf.predict(Xtest)

Best parameters set found on train set:

{'var_smoothing': 0.01}


In [ ]:
print(confusion_matrix(ytest, y_pred_nb,normalize='true'))

[[0.82       0.12       0.06      ]
 [0.32142857 0.60714286 0.07142857]
 [0.5        0.25       0.25      ]]


In [ ]:
f1_model2 = f1_score(ytest, y_pred_nb, average='macro')
print(f1_model2)

0.5677381784300023


In [ ]:
cohen_kappa_model2 = cohen_kappa_score(ytest, y_pred_nb)
print(cohen_kappa_model2)

0.41689008042895437


There is slight difference between the two models but it seems that the classification with naive bayes perform a little better then the one with AdaBoost